In [1]:
import numpy as np
import matplotlib.pyplot as plt
import batman
import emcee
from astropy.time import Time
import h5py
import corner
from matplotlib.ticker import ScalarFormatter
from mc3.stats import time_avg
import os
import gc

In [2]:
# All instrumental noise and astrophysical models
def transit_model(time, t_s, fp):
    # set up the parameters, only fp and t_secondary are fitted
    params = batman.TransitParams()       # object to store transit parameters
    params.t0 = 58388.7939*24             # time of mid transit in hours
    params.per = 3.777940*24              # orbital period in hours
    params.rp = 0.0539                    # planet radius (in units of stellar radii), from cadieux 2024
    params.a = 26.57                      # semi-major axis (in units of stellar radii)
    params.inc = 89.8                     # orbital inclination (in degrees), from nasa exoplanet archive
    params.ecc = 0.                       # eccentricity
    params.w = 90.                        # longitude of periastron (in degrees)
    params.limb_dark = "quadratic"        # limb darkening model
    params.u = [0.16, 0.22]               # limb darkening coefficients, values from cadieux 2024b
    params.fp = fp                        # planet to star flux ratio
    params.t_secondary = t_s

    # generate the light curve model
    m = batman.TransitModel(params, time, transittype="secondary")
    flux = m.light_curve(params)
    return flux

# instrumental noise models, all as a function of time
def exponential_func(time, a, b, c):
    return a * np.exp(-b * time) + c

def linear_slope(time, m, b):
    return m*time + b

def poly_3rd_degree(t, a, b, c, d):
    return a * t**3 + b * t**2 + c * t + d

def double_exponential(time, c1, c2, c3, c4):
    return c1 * np.exp(-c2 * time) + c3 * np.exp(-c4 * time)

# instrumental second-degree polynomial centroid models, as a function of centroid_x and centroid_y
def detec_model_poly(xdata, ydata, c1, c2, c3, c4, c5, c6):
    x, y = xdata, ydata
    pos = np.vstack((np.ones_like(x),
                     x   ,      y,
                     x**2, x   *y,      y**2))
    detec = np.array([c1, c2, c3, c4, c5, c6])
    return np.dot(detec[np.newaxis,:], pos).reshape(-1)

# Redirect to each model 
# For many models, they are multiplied with each other
def detec_model(time, centroid_x, centroid_y, theta, model_type):
    detec = []
    centroid = False
    
    if '2nd_order_centroid' in model_type:
        detec_centroid = detec_model_poly(centroid_x, centroid_y, theta[2], theta[3], theta[4], theta[5], theta[6], theta[7])
        theta = np.delete(theta, np.s_[2:8])
        model_type = model_type.replace('_2nd_order_centroid', '')  # Remove for processing
        centroid = True

    if model_type == 'exp+linear':
        t_s, fp, c1, c2, c3, c4, c5, sigF = theta
        detec = exponential_func(time, c1, c2, c3)*linear_slope(time, c4, c5)
    elif model_type == 'exp':
        t_s, fp, c1, c2, c3, sigF = theta
        detec = exponential_func(time, c1, c2, c3)
    elif model_type == 'linear':
        t_s, fp, c1, c2, sigF = theta
        detec = linear_slope(time, c1, c2)
    elif model_type == 'polynomial':
        t_s, fp, c1, c2, c3, c4, sigF = theta
        detec = poly_3rd_degree(time, c1, c2, c3, c4)
    elif model_type == 'exp+polynomial':
        t_s, fp, c1, c2, c3, c4, c5, c6, c7, sigF = theta
        detec = exponential_func(time, c1, c2, c3)*poly_3rd_degree(time, c4, c5, c6, c7)
    elif model_type == 'linear+polynomial':
        t_s, fp, c1, c2, c3, c4, c5, c6, sigF = theta
        detec = linear_slope(time, c1, c2)*poly_3rd_degree(time, c3, c4, c5, c6)
    elif model_type == 'double_exp':
        t_s, fp, c1, c2, c3, c4, sigF = theta
        detec = double_exponential(time, c1, c2, c3, c4)

    # Multiply the instrumental noise model with the centroid model
    if centroid:
        detec = detec * detec_centroid
    return detec

def signal(time, centroid_x, centroid_y, theta, model_type):
# multiply the astrophysical model with the instrumental noise model to retrieve the signal
    t_s = theta[0]
    fp = theta[1]
    astro = transit_model(time, t_s, fp)
    detec = detec_model(time, centroid_x, centroid_y, theta, model_type)
    return astro*detec

In [16]:
def set_p0_detec(model_type, ecl_nb):
    p0_detec = []
    if ecl_nb == '1':
        if 'exp' in model_type:
                p0_detec += [0.0034727281160956787, 2.180515561003361, 0.9993508344634218]
        if 'double_exp' in model_type:
            p0_detec = [0.9996186568147779, 0.00010717988935901778, 0.003320590954562985, 2.786865103285596]
        if 'linear' in model_type:
            p0_detec += [ -0.00047958976911880607, 1.0004812673893524]
        if 'polynomial' in model_type:
            p0_detec += [-0.00023014272610047499, 0.0016926563548700275, -0.003946951506184123, 1.002279224651203]
        if '2nd_order_centroid' in model_type:
            p0_detec = np.append([0.5, 0.1, 0.1, -0.1, 0.1, 0.1], p0_detec)

    elif ecl_nb == '2':
        if 'exp' in model_type:
            p0_detec += [0.21210932148849349, 0.001098500480854919, 0.7882074156441597]
        if 'double_exp' in model_type:
            p0_detec = [1.000393934067105, 0.00027358732870096445, -0.001123853891476274, 8.878490746867785]
        if 'linear' in model_type:
            p0_detec += [-0.00021872009640224572, 1.0004497232626075]
        if 'polynomial' in model_type:
            p0_detec += [6.398419732442172e-05, -0.00042333685581228634,  0.000550183940545113,  0.999979854087533]
        if '2nd_order_centroid' in model_type:
            p0_detec = np.append([0.5, 0.1, 0.1, -0.1, 0.1, 0.1], p0_detec)

    elif ecl_nb == '3':
        if 'exp' in model_type:
            p0_detec += [ 0.5202747034072548, 0.0001764227142003564, 0.4797072071658882]
        if 'double_exp' in model_type:
            p0_detec = [1.0479772630784343, -0.0003293269669427706, -0.04799803531595613, -0.008853702122369113]
        if 'linear' in model_type:
            p0_detec += [-8.74165359321118e-05,  0.999957647367997]
        if 'polynomial' in model_type:
            p0_detec += [4.4553644530141374e-05, -0.0002559721521483956, 0.0003018805949031837, 0.9998792766272656]
        if '2nd_order_centroid' in model_type:
            p0_detec = np.append([0.5, 0.1, 0.1, -0.1, 0.1, 0.1], p0_detec)

    return p0_detec

def set_p0_astro(eclipse, fp):
    p0_astro = [eclipse, fp]
    return p0_astro

In [ ]:
# calculate chi-squared
def chi_squared(aplev, astro, detec, p0_mcmc):    
    sigF = p0_mcmc[-1]
    sigma2 = sigF**2
    return np.sum((aplev - astro*detec) ** 2 / sigma2)

# calculate BIC
def BIC(n_dat, n_par, lnL):
    return n_par*np.log(n_dat) - 2*lnL  

## RUN MCMC

In [4]:
# MCMC functions
def log_likelihood(theta, time, flux, centroid_x, centroid_y, model_type):
    model = signal(time, centroid_x, centroid_y, theta, model_type)
    sigF = theta[-1]
    sigma2 = sigF**2
    return -0.5 * np.sum((flux - model) ** 2 / sigma2 + np.log(sigma2))

def log_prior(theta,model_type, eclipse):
    prior = 0.
    t_s = theta[0]
    fp = theta[1]
    sigF = theta[-1]
    e_dur = 1.13 # hours
    
    # set up the priors for the parameters
    if (0 < fp < 1 and (eclipse-(e_dur/2)) < t_s < (eclipse+(e_dur/2)) and sigF > 0. ): 
        return 0.0 + prior
    return np.inf

def log_prob(theta, time, flux, centroid_x, centroid_y, model_type, eclipse):
    lp = log_prior(theta, model_type, eclipse)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, time, flux, centroid_x, centroid_y, model_type)

In [ ]:
def binValues(values, binAxisValues, nbin, assumeWhiteNoise=True):
    bins = np.linspace(np.nanmin(binAxisValues), np.nanmax(binAxisValues), nbin)
    digitized = np.digitize(binAxisValues, bins)
    binned = np.array([np.nanmedian(values[digitized == i]) for i in range(1, nbin)])
    binnedErr = np.nanmean(np.array([np.nanstd(values[digitized == i]) for i in range(1, nbin)]))
    if assumeWhiteNoise:
        binnedErr /= np.sqrt(len(values)/nbin)
    return binned, binnedErr

# bin and retrieve proper errorbars on aplev
def binFluxError(values, error, binAxisValues, nbin, assumeWhiteNoise=True):
    bins = np.linspace(np.nanmin(binAxisValues), np.nanmax(binAxisValues), nbin)
    digitized = np.digitize(binAxisValues, bins)
    binned = np.array([np.nanmedian(values[digitized == i]) for i in range(1, nbin)])
    binCounts = np.array([np.sum(digitized == i) for i in range(1, nbin)])

    binnedErr = np.array([np.mean(error[digitized == i]) for i in range(1, nbin)])/np.sqrt(binCounts)
    return binned, binnedErr

In [4]:
def save_chains(savepath, chain, lnprobchain, pos2, prob):
    chain_savepath = '/Volumes/HardDrive/new_extraction_fortune_2025/'+savepath
    #chain_savepath = '/Volumes/HardDrive/LHS1140-c/photom_etienne/'+savepath
    os.makedirs(chain_savepath, exist_ok=True)

    pathchain = chain_savepath + 'samplerchain_10000.npy'
    pathlnlchain = chain_savepath + 'samplerlnlchain_10000.npy'
    pathposit = chain_savepath + 'samplerposi_10000.npy'
    pathlnpro = chain_savepath + 'samplerlnpr_10000.npy'
    np.save(pathchain, chain)
    np.save(pathlnlchain, lnprobchain)
    np.save(pathposit, pos2)
    np.save(pathlnpro, prob)

def print_mcmc_results(p0_mcmc, MCMC_Results):
    
    for i in range(len(p0_mcmc)):
        MCMC_Results[i] = (p0_mcmc[i], MCMC_Results[i][1], MCMC_Results[i][2])

    p0_labels = ["t_s", "fp"] + [f"c_{i}" for i in range(1, len(p0_mcmc) - 2)] + ["sigF"]

    out = "MCMC result:\n\n"
    for i in range(len(p0_mcmc)):
        out += '{:>8} = {:>16}  +{:>16}  -{:>16}\n'.format(p0_labels[i],MCMC_Results[i][0], MCMC_Results[i][1], MCMC_Results[i][2])
    print(out, flush=True)
    return out, p0_labels

def calculate_detec_model(model_type, p0_mcmc, time, centroid_x, centroid_y):
    p0_centroid = p0_mcmc
    model_no_centroid = model_type
    centroid = False
    if '2nd_order_centroid' in model_type:
        detec_centroid = detec_model_poly(centroid_x, centroid_y, p0_centroid[2], p0_centroid[3], p0_centroid[4], p0_centroid[5], p0_centroid[6], p0_centroid[7])
        p0_centroid = np.delete(p0_centroid, np.s_[2:8])
        model_no_centroid = model_type.replace('_2nd_order_centroid', '')  # Remove for processing
        centroid = True

    if model_no_centroid == 'exp+linear':
        t_s, fp, c1, c2, c3, c4, c5, sigF = p0_centroid
        detec = detec_model(time, centroid_x, centroid_y, [t_s, fp, c1, c2, c3, c4, c5, sigF], model_no_centroid)
    elif model_no_centroid == 'exp':
        t_s, fp, c1, c2, c3, sigF = p0_centroid
        detec = detec_model(time, centroid_x, centroid_y, [t_s, fp, c1, c2, c3, sigF], model_no_centroid)
    elif model_no_centroid == 'linear':
        t_s, fp, c1, c2, sigF = p0_centroid
        detec = detec_model(time, centroid_x, centroid_y,[t_s, fp, c1, c2, sigF], model_no_centroid)
    elif model_no_centroid == 'polynomial':
        t_s, fp, c1, c2, c3, c4, sigF = p0_centroid
        detec = detec_model(time, centroid_x, centroid_y, [t_s, fp, c1, c2, c3, c4, sigF], model_no_centroid)
    elif model_no_centroid == 'exp+polynomial':
        print('including exp+poly model')
        t_s, fp, c1, c2, c3, c4, c5, c6, c7, sigF = p0_centroid
        theta = [t_s, fp, c1, c2, c3, c4, c5, c6, c7, sigF]
        print(type(theta))
        detec = detec_model(time, centroid_x, centroid_y, [t_s, fp, c1, c2, c3, c4, c5, c6, c7, sigF], model_no_centroid)

    elif model_no_centroid == 'linear+polynomial':
        t_s, fp, c1, c2, c3, c4, c5, c6, sigF = p0_centroid
        detec = detec_model(time, centroid_x, centroid_y, [t_s, fp, c1, c2, c3, c4, c5, c6, sigF], model_no_centroid)
    elif model_no_centroid == 'double_exp':
        t_s, fp, c1, c2, c3, c4, sigF = p0_centroid
        detec = detec_model(time, centroid_x, centroid_y, [t_s, fp, c1, c2, c3, c4, sigF], model_no_centroid)
    

    if centroid:
        detec = detec * detec_centroid
        print('including centroid model')

    return detec

def calculate_astro_model(time, t_s, fp):
    astro = transit_model(time, t_s, fp)
    return astro

def make_result_file(out, time, aplev, astro, detec, residuals, p0_mcmc, lnprobchain, savepath):

    # calculate RMS
    rms = np.std(residuals)
    print('RMS = ', rms*1e6, 'ppm')  

    # # # using convolution dont think this is right?
    # conv = np.convolve(aplev, residuals, mode='full')
    # rms2 = np.std(conv)
    # print('RMS2 = ', rms2)

    # calculate chi2
    #chi2 = np.sum((residuals)**2 / (aperr)**2) # alternaive calculation?
    chi = chi_squared(aplev, astro, detec, p0_mcmc)
    print('chi2 = ', chi)

    # calculate BIC
    bic = BIC(len(time), len(p0_mcmc), lnprobchain.max()) # lnL is the likelihood value of best fit model, formula that depends on chi2, recheck
    print('BIC:', bic)

    # write to a text file with the same naming convention as the saved chains
    os.makedirs(savepath, exist_ok=True)
    textfile = savepath+'results.txt'
    
    with open(textfile, 'w') as f:
        f.write(out) # MCMC output
        f.write('RMS = '+str(rms*1e6)+' ppm\n')
        f.write('chi2 = '+str(chi)+'\n')
        f.write('BIC = '+str(bic)+'\n')
        f.close()
    

In [25]:
def plot_corrected_flux(time, aplev, aperr, errorbars, detec, astro, residuals, t_s, eclipse, savepath):
    fig, ax = plt.subplots(2, 1, figsize=(12, 10),  gridspec_kw={'height_ratios': [3, 1]})
    ax[0].errorbar(time, aplev, yerr=aperr, label='Data', color='grey', alpha=0.1, marker='o', zorder=1)
    ax[0].errorbar(time, aplev/detec, yerr=errorbars, label='Corrected data', color='red', alpha=0.2, marker='o', zorder=1)
    ax[0].plot(time, astro, label='Astro Model',color = 'black', zorder=2, linewidth=3)
    ax[0].plot(time, detec, label='Detec Model', color = 'grey', zorder=2, linewidth=3)
    ax[0].vlines(t_s, 0.994, 1.008, color='blue', label='t_s', zorder=3)
    ax[0].vlines(eclipse, 0.994, 1.008, color='green', label='eclipse', zorder=3)
    ax[0].set_ylim(0.9968, 1.005)
    ax[1].set_ylim(-0.003, 0.003)
    ax[0].set_xlim(0, 3.8)
    ax[1].set_xlim(0, 3.8)
    ax[0].legend()
    ax[1].scatter(time,residuals, label='Residuals', color='black', alpha=0.1, marker='o', s=3)
    ax[1].axhline(y=0, color='r', linewidth = 2)
    ax[0].set_ylabel('Normalized Flux')
    ax[1].set_ylabel('Residuals')
    #hide y axis
    ax[0].set_xticks([])  # Remove x-axis ticks
    ax[0].set_xlabel('')  # Remove x-axis label

    # no h space
    plt.subplots_adjust(hspace=0.0)
    plt.xlabel('Time (hours since begining of observations)')
  
    plt.savefig(savepath+'corrected_flux.png', bbox_inches='tight')
    plt.show()
    plt.close()

def plot_rednoise(residuals, minbins, occDuration = None, showPlot=True, fname=None, fontsize=10):
    
    maxbins = int(np.rint(residuals.size/minbins))
    
    try:
        rms, rmslo, rmshi, stderr, binsz = time_avg(residuals, maxbins)
    except:
        rms = []
        for i in range(minbins,len(residuals)):
            rms.append(helpers.binnedNoise(np.arange(len(residuals)),residuals,i))
        rms = np.array(rms)[::-1]

        binsz = len(residuals)/np.arange(minbins,len(residuals))[::-1]

        #In case there is a NaN or something while binning
        binsz = binsz[np.isfinite(rms)]
        rms = rms[np.isfinite(rms)]
        rmslo = np.zeros_like(rms)
        rmshi = rmslo
        stderr = np.std(residuals)/np.sqrt(binsz)
    
    plt.clf()
    ax = plt.gca()
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.errorbar(binsz, rms, yerr=[rmslo, rmshi], fmt="-", color='#214C6C', ecolor='#B1D0E7', capsize=0, linewidth=2, label="Data RMS")
    ax.plot(binsz, stderr, c='k', label="Gaussian std.")
    ylim = ax.get_ylim()
    if occDuration is not None:
        ax.plot([occDuration,occDuration],ylim, color='black', ls='-.', alpha=0.6)
    ax.set_ylim(ylim)
    
    ax.xaxis.set_tick_params(labelsize=fontsize)
    ax.yaxis.set_tick_params(labelsize=fontsize)
    
    plt.xlabel(r'N$_{\rm binned}$', fontsize=fontsize)
    plt.ylabel('RMS', fontsize=fontsize)
    plt.legend(loc='best', fontsize=fontsize)
    if fname is not None:
        plt.savefig(fname)

    if showPlot:
        plt.show()
    
    plt.close()
    
def plot_walker(chain, labels, interv=10, fname=None, showPlot=False, fontsize=15):
    nwalk = chain.shape[0]
    ndim = chain.shape[-1]

    # get first index
    beg   = 0
    end   = len(chain[0,:,0])
    step  = np.arange(beg,end)
    step  = step[::interv]

    # number of columns and rows of subplots
    ncols = 4
    nrows = int(np.ceil(ndim/ncols))
    sizey = 2*nrows

    # plotting
    plt.figure(figsize = (15, 2*nrows))
    for ind in range(ndim):
        plt.subplot(nrows, ncols, ind+1)
        sig1 = (0.6827)/2.*100
        sig2 = (0.9545)/2.*100
        sig3 = (0.9973)/2.*100
        percentiles = [50-sig3, 50-sig2, 50-sig1, 50, 50+sig1, 50+sig2, 50+sig3]
        neg3sig, neg2sig, neg1sig, mu_param, pos1sig, pos2sig, pos3sig = np.percentile(chain[:,:,ind][:,beg:end:interv],
                                                                                       percentiles, axis=0)
        plt.plot(step, mu_param)
        plt.fill_between(step, pos3sig, neg3sig, facecolor='k', alpha = 0.1)
        plt.fill_between(step, pos2sig, neg2sig, facecolor='k', alpha = 0.1)
        plt.fill_between(step, pos1sig, neg1sig, facecolor='k', alpha = 0.1)
        plt.title(labels[ind], fontsize=fontsize)
        plt.xlim(np.min(step), np.max(step))
        if ind < (ndim - ncols):
            plt.xticks([])
        else:
            plt.xticks(rotation=25)

        y_formatter = ScalarFormatter(useOffset=False)
        plt.gca().yaxis.set_major_formatter(y_formatter)
        plt.gca().xaxis.set_tick_params(labelsize=fontsize*0.8)
        plt.gca().yaxis.set_tick_params(labelsize=fontsize*0.8)

    if fname != None:
        plt.savefig(fname, bbox_inches='tight')

    if showPlot:
        plt.show()
    plt.close()
    return

def plot_binned(time, astro, bins_time, bins_aplev, bins_error, bins_astro, bins_detec, t_s_minus_err, t_s_plus_err, eclipse, savepath):

    fig, ax = plt.subplots(2, 1, figsize=(8, 6),  gridspec_kw={'height_ratios': [2, 1]})
    ax[0].errorbar(bins_time, bins_aplev/bins_detec, yerr=bins_error, label='Corrected data', color='red', linestyle='none', alpha=0.2, marker='o', zorder=1)
    ax[0].plot(time, astro, label='Astro Model',color = 'black', zorder=2, linewidth=3)
    ax[0].vlines(eclipse, 0.994, 1.008, color='black', label='expected eclipse', zorder=3)
    ax[0].axvspan(t_s_minus_err, t_s_plus_err, color='red', alpha=0.3, label = 't_s uncertainty')
    ax[0].set_ylim(0.999, 1.0015)
    ax[0].set_xlim(0,3.75)
    ax[1].set_xlim(0,3.75)
    ax[0].legend()
    res = (bins_aplev/bins_detec)-bins_astro  ## should residuals be binned instead of re-calculated
    ax[1].errorbar(bins_time,res,yerr=bins_error,  label='Residuals', color='black', alpha=0.1, marker='o', linestyle='none')
    ax[1].axhline(y=0, color='r', linewidth = 2)
    fig.subplots_adjust(hspace=0)
    plt.xlabel('Time (hours since begining of observations)')
    ax[0].set_ylabel('Normalized Flux') 
    plt.tight_layout()  
    plt.savefig(savepath+'binned.png')
    plt.close()
    return 

def plot_corner(chain, p0_labels, savepath, showPlot=True, fontsize=15):
    fig = corner.corner(chain[:, :2], quantiles=[0.16, 0.5, 0.84], labels = p0_labels, show_titles=True,
                        plot_datapoints=False, label_kwargs={"fontsize": 24}, title_kwargs={"fontsize": 18}, quiet=False, title_fmt=".5f", )
    if savepath != None:
        plt.savefig(savepath+'corner.png', bbox_inches='tight')
    if showPlot:
        plt.show()
    plt.close()
    return


In [24]:
def run_mcmc(model_type, time, aplev, aperr, centroid_x, centroid_y, eclipse, fp, savepath, ecl_nb, nstep = 100000, p0_detec = None, p0_astro = None):
    # set first guess
    if p0_astro is None:
        p0_astro = set_p0_astro(eclipse, fp)
    if p0_detec is None:
        p0_detec = set_p0_detec(model_type, ecl_nb) 

    p0 = np.concatenate((p0_astro, p0_detec, [0.001]))
    nwalkers, ndim = 70, len(p0)
    pos = p0 + 1e-4 * np.random.randn(nwalkers, ndim)

    # Run the MCMC sampler
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=(time, aplev, centroid_x, centroid_y, model_type, eclipse))
    pos2, prob, state = sampler.run_mcmc(pos, nstep, progress=True)

    if nstep >= 100000:
        nBurnInSteps = 90000
    elif nstep == 1000:
        nBurnInSteps = 500
    else:
        nBurnInSteps = 5000
        
    lnprobchain = sampler.get_log_prob(discard=nBurnInSteps).swapaxes(0,1)
    chain = sampler.get_chain(discard=nBurnInSteps).swapaxes(0,1)

    # Save the chains
    save_chains(savepath, chain, lnprobchain, pos2, prob)

    # Reformat and print out MCMC results
    samples = chain.reshape((-1, ndim))
    MCMC_Results = np.array(list(map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84],axis=0)))))
    maxk, maxiter = np.unravel_index((lnprobchain).argmax(), (lnprobchain).shape)
    p0_mcmc = chain[maxk, maxiter,:]
    out, p0_labels = print_mcmc_results(p0_mcmc, MCMC_Results)

    print('t_s in minutes is ', (p0_mcmc[0]-eclipse))

    
    # calculate the models and bin the data
    detec = calculate_detec_model(model_type, p0_mcmc, time, centroid_x, centroid_y)
    astro = calculate_astro_model(time, p0_mcmc[0], p0_mcmc[1])
    residuals = (aplev/detec)-astro

    nbin = 100
    errorbars = np.full_like(aplev, p0_mcmc[-1], dtype=float)
    bins_time, err = binValues(time, time, nbin)
    bins_aplev, err = binValues(aplev, time, nbin)
    bins_detec, err = binValues(detec, time, nbin) ## CHECK if can simply bin model
    bins_astro, bins_error = binFluxError(astro, errorbars, time, nbin)

    # Save the results (add)
    make_result_file(out, time, aplev, astro, detec, residuals, p0_mcmc, lnprobchain, savepath)

    # Make some plots
    plot_corrected_flux(time, aplev, aperr, errorbars, detec, astro, residuals, p0_mcmc[0], eclipse, savepath)
    plot_rednoise(residuals, 3, fname=savepath+'rednoise.png', occDuration=366)
    plot_walker(chain, p0_labels, interv=100, fname=savepath+'walkers.png', showPlot=True, fontsize=15)
    plot_binned(time, astro, bins_time, bins_aplev, bins_error, bins_astro, bins_detec, p0_mcmc[0]-MCMC_Results[0][2], p0_mcmc[0]+MCMC_Results[0][1], eclipse, savepath)
    plot_corner(samples, p0_labels, savepath)

    del sampler, chain, lnprobchain, samples, MCMC_Results, p0_mcmc, out, p0_labels
    gc.collect()
    

In [ ]:
# plt.rc('xtick', labelsize=12) 
# plt.rc('ytick', labelsize=12) 

